In [2]:
import pandas as pd
import os

In [13]:
umls = pd.read_csv('umls/mrconso.txt',sep='|')
mrsty = pd.read_csv('umls/mrsty.txt',sep='|')
terminology = pd.read_table('../ncbi-data/TERMINOLOGY.txt',sep='\|\|',header=None, names=['cui','name'])
terminology.head()

C:\Users\etfrench\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,cui,name
0,C565588,Epidermolysis Bullosa With Diaphragmatic Hernia
1,C567755|613097,"Tooth Agenesis, Selective, 6|STHAG6"
2,C565584,"Epithelial Squamous Dysplasia, Keratinizing De..."
3,C565585,"Epiphyseal Dysplasia of Femoral Head, Myopia, ..."
4,C565587,"Epilepsy, Photogenic, with Spastic Diplegia an..."


In [151]:
#Load a dataset of all training annotations
train_path = '../n2c2-data/all'
files = os.listdir(train_path)
train = pd.DataFrame([])
for file in files:
    if '.concept' in file:
        df = pd.read_table(f'{train_path}/{file}',sep='\|\|',header=None, names=['file_id','ix','type','name','CUI'])
        df['file'] = [file for x in range(len(df))]
        train = pd.concat([train, df])
train = train[['CUI','name','file']]
train = pd.merge(train, mrsty, on='CUI')[['CUI','name','file','TUI','STY']]
train_cuis = list(set(train.CUI))
train.head()

C:\Users\etfrench\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,CUI,name,file,TUI,STY
0,C0564823,Right LE pain,0034.concept,T184,Sign or Symptom
1,C0564823,R leg pain,0034.concept,T184,Sign or Symptom
2,C0564823,Right LE pain,0034.concept,T184,Sign or Symptom
3,C0030193,Pain,0034.concept,T184,Sign or Symptom
4,C0030193,Pain,0034.concept,T184,Sign or Symptom


In [12]:
# Load MRCONSO from source
file = 'umls/MRCONSO.RRF'
# cols = ['CUI','TUI','STN','STY','ATUI','CVF','BLANK']
cols = ['CUI','LAT','TS','LUI','STT','SUI','ISPREF','AUI','SAUI','SCUI','SDUI','SAB','TTY','CODE','STR','SRL','SUPPRESS','CVF','BLANK']
umls = pd.read_table(file,sep='|',header=None,names=cols)
umls = umls.drop(columns=['BLANK'])
print(len(umls))
umls = umls[umls.LAT=='ENG']
print(len(umls))
umls = umls[(umls.SAB=='RXNORM') | (umls.SAB=='SNOMEDCT_US')]
print(len(umls))
umls.to_csv('umls/mrconso.txt',sep='|',index=False)
umls.head()

C:\Users\etfrench\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


16132273
11434670
1850431


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
9,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0
19,C0000039,ENG,S,L0012507,PF,S0033298,N,A22817493,166113012.0,102735002,NaN,SNOMEDCT_US,OAP,102735002,Dipalmitoylphosphatidylcholine,9,O,256.0
25,C0000039,ENG,S,L3000054,PF,S3260062,Y,A22880204,544223010.0,102735002,NaN,SNOMEDCT_US,OAF,102735002,Dipalmitoylphosphatidylcholine (substance),9,O,NaN
61,C0000052,ENG,P,L0000052,VC,S0575717,Y,A27769867,97197014.0,58488005,NaN,SNOMEDCT_US,PT,58488005,"1,4-alpha-Glucan branching enzyme",9,N,NaN
69,C0000052,ENG,S,L0006129,VC,S0604824,Y,A27781005,97198016.0,58488005,NaN,SNOMEDCT_US,SY,58488005,Branching enzyme,9,N,256.0


In [124]:
# Code to derive N2C2_TUI
# Get distinct TUI list in train order by # of CUIs in UMLS
tuis = pd.merge(mrsty, pd.DataFrame(set(train_cui_tui.TUI), columns=['TUI']), on='TUI') \
    .groupby('TUI').count()['CUI'] \
    .reset_index(name='count') \
    .sort_values(['count'], ascending=False) \
    .TUI.tolist()

expected = len(set(train_cui_tui.CUI))
removed = []

for tui in tuis:
    # Try removing records with given TUI
    subset = train_cui_tui[(train_cui_tui.TUI.isin(tuis)) & (train_cui_tui.TUI != tui)]
    
    # If removing the TUI didn't drop unique CUIs, remove it permanently
    if expected == len(set(subset.CUI)):
        tuis.remove(tui)
        
n2c2_tui = train_cui_tui[(train_cui_tui.TUI.isin(tuis))] \
    .groupby('TUI').count()['CUI'] \
    .reset_index(name='count') \
    .sort_values(['count'], ascending=False)

# Used for SQL insert statement
# for x in n2c2_tui.iterrows():
#     tui = x[1][0]
#     count = x[1][1]
#     print(f"('{tui}',{count}),")